In [26]:
import pandas as pd
import os
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import wordninja
import re
import collections

download('stopwords')  # Download stopwords list.
path_to_model = "../models/"
stopwords = stopwords.words("english")


def get_voc_of_model(model_df, stopwords, remove_stop_word=True, lemmatized_word=True):
    '''
    :param model_df: dataframe contenant le model
    :param stopwords: nltk.corpus contenant les stopwords à enlever
    :param remove_stop_word: Bool : permet de choisir d'enlever les stop words (True) ou de les laisser (False)
    :param lemmatized_word: Bool : permet de choisir de prendre la forme lemmatizé du vocabulaire (True) ou de les laisser tels quels (False)
    :return: Liste des mots du vocabulaire
    '''
    voc= []
    if lemmatized_word:
        lematizer = WordNetLemmatizer()
    # Récupération des clés
    for key_path in model_df["Unnamed: 0"]:
        # print(key_path)
        # Récupération de chaque noeud
        for label in key_path.split("."):
            # Split selon le camel case
            camel_case_split_of_label = [match.group(0) for match in re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', label)]

            for splited_label in camel_case_split_of_label:
                # Split selon une distribution statistiques si les mots n'ont pas été différenciés par Camelcase
                for word in wordninja.split(splited_label):
                    # Soit on garde, soit on ne garde pas les stop words (ex : for, or, in, the...)
                    if lemmatized_word:
                        word_lemmatized = lematizer.lemmatize(word)
                        if remove_stop_word:
                            if word not in stopwords:
                                voc.append(word_lemmatized)
                        else:
                            voc.append(word_lemmatized)
                    else:
                        if remove_stop_word:
                            if word not in stopwords:
                                voc.append(word)
                        else:
                            voc.append(word)
    return voc

def voc_complexity(voc, word_frequency_doc, num_occur):
    '''
    :param voc: vocabulaire (liste des mots distincts)
    :param word_frequency_doc: dataframe contennant la liste des mots de la langue anglaise (récupérés depuis wikipédia) et leur nombre d'occurange
    :param num_occur: Somme des occurance de mots pour le nombre total de mots
    :return: res : dictionnaire contenant les mots du vocabulaires associés à leur fréquence
    + le clé "Not_a_word" qui contient le nombre total de mots du vocabulaire n'étant pas un mot de la langue anglaise
    (peut être : une erreur de récupération des mots / splits des mots dans les labels - un acronyme - un mot qui n'est pas un mot)
    '''
    res = {"Not_a_word":[]}
    for word in voc:
        if word == "null":
            print(word)
        freq = word_frequency_doc[word_frequency[0] == word.lower()][1]
        if freq.size ==1:

           res[word]=freq.item()/num_occur
        else:
            res["Not_a_word"].append(word)
            # res["Not_a_word"]+=1
    return res

def get_average_frequency(voc_frequency):
    '''
    Retourn la moyenne des fréquences de mots du vocabulaire
    :param voc_frequency: dict : les mots et leur fréquence associée
    :return: float
    '''
    sum = 0
    for key in voc_frequency:
        if key !="Not_a_word":
            sum += voc_frequency[key]
    return sum/len(voc_frequency.keys())

def get_weithed_mean_frequency(voc_frequency, all_words):
    '''
    Retourne la moyenne pondérées (par les occurences des mots) des fréquences de mots du vocabulaire
    :param voc_frequency: dict : vocabulaire distinct et fréquence des mots associés
    :param all_words: list : liste totale des mots
    :return: float
    '''
    sum = 0
    # print(all_words)
    word_occur={}
    sum_of_occur = 0
    for word in all_words:
        if word in word_occur:
            word_occur[word]+=1
        else:
            word_occur[word]=1
        sum_of_occur+=1
    for key in voc_frequency:
        if key !="Not_a_word":
            sum += voc_frequency[key]*word_occur[key]
    return sum/sum_of_occur

def get_path_length(model_df):
    '''
    Retourne la taille des chemins et le nombre d'occurence de cette taille dans le model
    :param model_df: Dataframe : contient le modèle
    :return: dict : dictionnaire contenant en clé la taille et en valeur le nombre d'occurence de cette taille dans les chemins du modèle
    '''
    res={}
    for key_path in model_df["Unnamed: 0"]:
        len_key_path=len(key_path.split("."))
        if len_key_path in res:
            res[len_key_path]+=1
        else:
            res[len_key_path]=1
    res = dict(sorted(res.items()))
    # return dict(collections.OrderedDict(sorted(res.items()))
    return res

def get_node_number(model):
    '''
    A REVOIR : retourne la taille du nombre de noeuds, à revoir car ne marche pas correctement
    :param model:
    :return:
    '''
    nodes=[]
    # print(model["Unnamed: 0"])
    for path in model["Unnamed: 0"]:
        splitted = path.split(".")
        for index,node in enumerate(splitted):
            nodes.append(".".join(splitted[:index+1]))
    # print("coucocu")
    # print(nodes)
    nodes = set(nodes)
    # print(nodes)
    return len(nodes)

def get_child_distrib(model_df):
    child_list = {}
    for path in model_df["Unnamed: 0"]:
        for index in range(len(path.split(""))):
            # child_list[:index+1]
            pass

    #TODO : Liste des fils de chaque noeud
    #TODO : Taille des listes de fils
    #TODO : Retourner le résultats / compte / max

res = {}
word_frequency=pd.read_csv("enwiki-2022-08-29.txt",delimiter=" ", header=None)
word_occur=word_frequency[1].sum()
for model_file in [model_path for model_path in list(os.walk(path_to_model))[0][2] if model_path.endswith(".csv")]:
    model_df = pd.read_csv(path_to_model+model_file)
    model_name = model_file.split("_")[0]
    res[model_name]={}
    model_voc_occur = get_voc_of_model(model_df, stopwords=stopwords, lemmatized_word=False)
    model_voc = set(model_voc_occur)
    res[model_name]["Longueur_voc"]=len(model_voc)
    lengths = get_path_length(model_df)
    res[model_name]["Path lengths distribution"]=lengths
    length_key_list = list(lengths.keys())
    len_length_key_list = len(length_key_list)
    res[model_name]["Path max length (model depth)"]=length_key_list[len_length_key_list-1]
    res[model_name]["Min length"]=length_key_list[0]
    res[model_name]["Number of path (number of leaf)"]=len(model_df["Unnamed: 0"])

    res[model_name]["Node number (leaf + intermediate node)"]= get_node_number(model_df)
    res[model_name]["Intermediate node"]=res[model_name]["Node number (leaf + intermediate node)"]-res[model_name]["Number of path (number of leaf)"]

    voc_complexity_var = voc_complexity(model_voc,word_frequency,word_occur)
    res[model_name]["Average word frequency"]= get_average_frequency(voc_complexity_var)
    res[model_name]["Weighted average word frequencies"] = get_weithed_mean_frequency(voc_complexity_var, model_voc_occur)
    # if model_name=="DublinCore":
    print(voc_complexity_var)
    print(len(voc_complexity_var))
    print("voc_complexity_var[\"Not_a_word\"] for " + model_name + ":" + str(voc_complexity_var["Not_a_word"]))

print(res)

[nltk_data] Downloading package stopwords to /home/vdang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'Not_a_word': [], 'Relation': 2.9037840338034408e-05, 'Publisher': 2.9597873452455293e-05, 'Title': 0.00030607207729965534, 'Creator': 1.570929892544045e-05, 'Subject': 9.92204336580155e-05, 'Format': 8.352840447619156e-05, 'Identifier': 2.751234631857664e-06, 'Rights': 0.00019412589851813619, 'Description': 9.593301460530261e-05, 'Language': 0.00022546719370711464, 'Type': 0.00015582777494230974, 'Source': 0.00011153828388467221, 'Coverage': 3.838323874079349e-05, 'Date': 0.00012183187344885168, 'Contributor': 1.2314149567310315e-05}
16
voc_complexity_var["Not_a_word"] for DublinCore:[]
null
{'Not_a_word': ['colgroup', 'null', 'cellspacing'], 'associated': 0.00012793056005170816, 'interpretation': 2.5689154813920104e-05, 'intended': 8.3582680813272e-05, 'target': 5.1733572925093985e-05, 'Device': 4.143052612036263e-05, 'Person': 0.00011987340228586587, 'patient': 2.5930520040179326e-05, 'Authoring': 1.2105267906424993e-06, 'organizer': 5.733966065050833e-06, 'playing': 0.000222588080

In [22]:
wordninja.split("cellspacing")

['cellspacing']